#  Pulse of the Nation: A Statistical Investigation into the Vital Signs of Armenian Healthcare

- Students: Anush Aghinyan, Yeva Avetisyan
- Supervisor: Gurgen Hovakimyan

## Needed libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import date

import re

In [3]:
today = date.today()

In [4]:
data = pd.read_csv('foryevacsv.csv')
# data = pd.read_csv('cleaned_data.csv')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250033 entries, 0 to 250032
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   N                    250033 non-null  int64 
 1   Insured person Code  250028 non-null  object
 2   Work sphere          250033 non-null  object
 3   Start Date           250033 non-null  object
 4   End Date             250033 non-null  object
 5   DOB                  250000 non-null  object
 6   Sex                  250033 non-null  object
 7   Region               250033 non-null  object
 8   Claim Code           250033 non-null  object
 9   Claim Status         249885 non-null  object
 10  Illness/ICD Code     249957 non-null  object
 11  Illness/Category     250033 non-null  object
 12  Claim Amount         250033 non-null  object
dtypes: int64(1), object(12)
memory usage: 24.8+ MB


# Data cleaning

## - datetime, DOB

In [6]:
# Drop null values
data.dropna(inplace = True)

In [7]:
# Convert 'Start date' column to datetime format
data['Start Date'] = pd.to_datetime(data['Start Date'])

In [8]:
# Convert 'End date' column to datetime format
data['End Date'] = pd.to_datetime(data['End Date'])

In [9]:
# Find contract longevity
data['Contract Longevity'] = data['End Date'] - data['Start Date']

In [10]:
# Get unique values
unique_values = np.unique(data['DOB'])

In [11]:
# Identify problematic values
problematic_values = [value for value in unique_values if not str(value).isdigit()]

In [12]:
# # Replace problematic values with NaN or some other placeholder
# data['DOB'] = data['DOB'].replace(problematic_values, np.nan)

# # takes a bit long to run

In [13]:
# # Convert 'DOB' column to datetime format after replacing problematic values
# data['DOB'] = pd.to_datetime(data['DOB'], errors = 'coerce', format = 'mixed')

In [14]:
def extract_year(date_str):
    match = re.search(r'\d{4}', date_str)
    if match:
        return match.group(0)
    else:
        return None

In [15]:
np.unique(data['DOB'])

array(['###############################################################################################################################################################################################################################################################',
       '1/1/1940', '1/1/1941', ..., '9/9/2019', '9/9/2021', '9/9/2022'],
      dtype=object)

In [16]:
data['BirthYear'] = data['DOB'].apply(extract_year)
data.dropna(subset=["BirthYear"], inplace = True)

In [17]:
data['BirthYear'] = data['BirthYear'].astype(int)

In [18]:
data = data[data['BirthYear']<= 2024]
data = data[data['BirthYear']>= 1914]

In [19]:
data['BirthYear'].describe()

count    249755.000000
mean       1988.324702
std          14.674593
min        1930.000000
25%        1981.000000
50%        1989.000000
75%        1996.000000
max        2024.000000
Name: BirthYear, dtype: float64

In [20]:
# Add a column Age that finds the age of each insured person
data['Age'] = 2024 - data["BirthYear"] # if it will drop an error use int(year)

In [21]:
data['Age'].describe()

count    249755.000000
mean         35.675298
std          14.674593
min           0.000000
25%          28.000000
50%          35.000000
75%          43.000000
max          94.000000
Name: Age, dtype: float64

## - removing unnecessary signs
## - removing whitespace

In [22]:
# Replace hyphens and multiple spaces with 0 in the 'Claim Amount' column
data["Claim Amount"] = data['Claim Amount'].replace("-", 0)
data["Claim Amount"] = data['Claim Amount'].replace(" -   ", 0)

In [23]:
# Convert Claim Amount to floating-point number
data['Claim Amount'] = data['Claim Amount'].astype("float")

# Check unique values of Sex
np.unique(data["Sex"])

# Standardize Sex column

# Check unique values of Sex again
np.unique(data["Sex"])

# -----

In [24]:
data.groupby(['Sex'])['Claim Amount'].sum()

Sex
 Անհայտ     1.274201e+08
 Արական     3.321621e+06
 Իգական     2.068147e+09
Արական      2.902535e+09
Իգական      2.179531e+09
Name: Claim Amount, dtype: float64

In [25]:
data.groupby(['Region'])['Claim Amount'].sum()

Region
 Անհայտ          625850287.0
 Արագածոտն          170136.0
 Արարատ             224262.0
 Արմավիր            168794.0
 Գեղարքունիք        181075.0
                    ...     
Փարաքար             275110.0
Փոքր Մանթաշ         100990.0
Քաջարան              82803.0
Քասախ               247000.0
Օձուն                26128.0
Name: Claim Amount, Length: 69, dtype: float64

## - checking unique values of Region, Claim Status, Work sphere, and Illness/Category columns
## - translating 

In [26]:
np.unique(data["Claim Status"])

array(['Canceled', 'Satisfied', 'Unsatisfied', 'Հայտի բացում',
       'Հայտի վերջնական կարգավորում', 'Նախնական'], dtype=object)

In [27]:
np.unique(data["Illness/Category"])

array(['Ալերգոլոգիա', 'Ակնաբուժություն', 'Այրվածքաբանություն', 'Անհայտ',
       'Անոթաբանություն', 'Ատամնաբուժություն', 'Գաստրոէնտերոլոգիա',
       'Գինեկոլոգիա', 'Դիմածնոտային վիրաբուժություն',
       'Ընդհանուր վիրաբուժություն', 'Ընտանեկան բժիշկ', 'Թունաբանություն',
       'Թոքաբանություն', 'Ծննդօգնություն', 'Հեմատոլոգիա', 'Հղիություն',
       'Հոգեբուժություն', 'Մանկաբուժություն', 'Մաշկաբանություն',
       'Նախնական զննում', 'Նեյրովիրաբուժություն', 'Նեոնատոլոգիա',
       'Ներզատաբանություն', 'Նեֆրոլոգիական', 'Նյարդաբանություն',
       'Ողնաշարաբանություն', 'Ուռուցքաբանություն', 'Ուրոլոգիա',
       'Պարտադիր բժշկական զննություն', 'Պարտադիր պրոֆ. զննում',
       'Պրոկտոլոգիա', 'Ռևմատոլոգիա', 'Սրտաբանություն', 'Վարակաբանություն',
       'Վնասվածքաբանություն', 'Տարեկան բուժ զննում', 'ՔԿԱ'], dtype=object)

In [28]:
np.unique(data['Region'])

array([' Անհայտ ', ' Արագածոտն ', ' Արարատ ', ' Արմավիր ',
       ' Գեղարքունիք ', ' Երևան ', ' ԼՂՀ ', ' Լոռի ', ' Կոտայք ',
       ' Շիրակ ', ' Սյունիք ', ' Վայոց Ձոր ', ' Տավուշ ', 'Աբովյան',
       'Ագարակաձոր', 'Ալավերդի', 'Աշտարակ', 'Առինջ', 'Արագածոտն',
       'Արարատ', 'Արենի', 'Արզնի', 'Արթիկ', 'Արմավիր', 'Արշալույս',
       'Բյուրեղավան', 'Գավառ', 'Գեղարքունիք', 'Գետազատ', 'Գետամեջ',
       'Գյումրի', 'Գորիս', 'Դալարիկ', 'Եղեգնաձոր', 'Եղվարդ', 'Երևան',
       'Թալին', 'ԼՂՀ', 'Լոռի', 'Լուսառատ', 'Կապան', 'Կարենիս', 'Կոտայք',
       'Հաղարծին', 'Հրազդան', 'Մասիս', 'Մարալիկ', 'Մարտունի', 'Մեծամոր',
       'Մրգաստան', 'Շիրակ', 'Չարենցավան', 'Ջերմուկ', 'Ջրաշեն', 'Ջրվեժ',
       'Ռինդ', 'Սյունիք', 'Սպիտակ', 'Վաղարշապատ(Էջմ.)', 'Վայոց Ձոր',
       'Վայք', 'Վանաձոր', 'Վեդի', 'Տավուշ', 'Փարաքար', 'Փոքր Մանթաշ',
       'Քաջարան', 'Քասախ', 'Օձուն'], dtype=object)

In [29]:
np.unique(data["Work sphere"])

array([' Առողջապահություն ',
       ' Գյուղատնտեսություն, անտառային տնտեսություն և ձկնորսություն ',
       ' Էլեկտրականության, գազի, գոլորշու և լավորակ օդի մատակարարում ',
       ' Կացության և հանրային սննդի կազմակերպում ', ' Կրթություն ',
       ' Հանքագործական արդյունաբերություն և բացահանքերի շահագործում ',
       ' Մասնագիտական, գիտական և տեխնիկական գործունեություն ',
       ' Մեծածախ և մանրածախ առևտուր. Ավտոմեքենաների և մոտոցիկլների նորոգում ',
       ' Մշակող արդյունաբերություն ', ' Մշակույթ, զվարճություն, հանգիստ ',
       ' Պետական կառավարում և պաշտպանություն ',
       ' Ջրամատակարարում, կոյուղի, թափոնների կառավարում և վերամշակում ',
       ' Վարչարարական և օժանդակ գործունեություն ',
       ' Տեղեկատվություն և կապ ',
       ' Փոխադրումներ և պահեստային տնտեսություն ', ' Ֆիզ․անձ/ԱՁ/անհայտ ',
       ' Ֆինանսական և ապահովագրական գործունեություն '], dtype=object)

In [30]:
np.unique(data["Sex"])

array([' Անհայտ ', ' Արական ', ' Իգական ', 'Արական', 'Իգական'],
      dtype=object)

In [31]:
# Dictionary for translating Claim Status
translation_dict = {
    "Հայտի վերջնական կարգավորում": "Final claim confirmation",
    "Նախնական": "Initial",
    "Հայտի բացում": "Claim creation",
    "Satisfied": "Satisfied",
    "Canceled": "Canceled",
    "Unsatisfied": "Unsatisfied"
}

data["Claim Status"] = data["Claim Status"].map(translation_dict)

In [32]:
# Dictionary for translating illness/category
translation_dict = {
    "Ալերգոլոգիա":"Allergology",
    "Ակնաբուժություն":"Ophthalmology",
    "Այրվածքաբանություն":"Dermatology",
    "Անհայտ":"Unknown",
    "Անոթաբանություն":"Angiology",
    "Ատամնաբուժություն":"Dentistry",
    "Գաստրոէնտերոլոգիա":"Gastroenterology",
    "Գինեկոլոգիա":"Gynecology",
    "Դիմածնոտային վիրաբուժություն":"Maxillofacial surgery",
    "Ընդհանուր վիրաբուժություն":"General surgery",
    "Ընտանեկան բժիշկ":"Family doctor",
    "Թունաբանություն":"Toxicology",
    "Թոքաբանություն":"Pulmonology",
    "Ծննդօգնություն":"Obstetrics",
    "Հեմատոլոգիա":"Hematology",
    "Հղիություն":"Pregnancy",
    "Հոգեբուժություն":"Psychiatry",
    "Մանկաբուժություն":"Pediatrics",
    "Մաշկաբանություն":"Dermatology",
    "Նախնական զննում":"Initial screening",
    "Նեյրովիրաբուժություն":"Neurosurgery",
    "Նեոնատոլոգիա":"Neonatology",
    "Ներզատաբանություն":"Endocrinology",
    "Նեֆրոլոգիական":"Nephrological",
    "Նյարդաբանություն":"Neurology",
    "Ողնաշարաբանություն":"Vertebrology",
    "Ուռուցքաբանություն":"Oncology",
    "Ուրոլոգիա":"Urology",
    "Պարտադիր բժշկական զննություն":"Mandatory medical examination",
    "Պարտադիր պրոֆ. զննում":"Mandatory examination",
    "Պրոկտոլոգիա":"Proctology",
    "Ռևմատոլոգիա":"Rheumatology",
    "Սրտաբանություն":"Cardiology",
    "Վարակաբանություն":"Epidemiology",
    "Վնասվածքաբանություն":"Traumatology",
    "Տարեկան բուժ զննում":"Annual medical examination",
    "ՔԿԱ":"ENT"
}

data["Illness/Category"] = data["Illness/Category"].map(translation_dict)

In [33]:
# Dictionary for translating Regions
translation_dict = {
    " Անհայտ ": "Yerevan", 
    " Երևան ": "Yerevan",
    "Երևան": "Yerevan",
    " Արագածոտն ":  "Aragatsotn",
    "Արագածոտն": "Aragatsotn",
    "Աշտարակ": "Aragatsotn",
    "Թալին": "Aragatsotn",
    " Արարատ ": "Ararat",
    "Արարատ": "Ararat",
    "Գետազատ": "Ararat",
    "Լուսառատ": "Ararat",
    "Մասիս": "Ararat",
    "Վեդի": "Ararat",
    " Արմավիր ": "Armavir",
    "Արմավիր": "Armavir",
    "Արշալույս": "Armavir",
    "Դալարիկ": "Armavir",
    "Մեծամոր": "Armavir",
    "Մրգաստան": "Armavir",
    "Վաղարշապատ(Էջմ.)": "Armavir",
    "Փարաքար": "Armavir",
    " Գեղարքունիք ": "Gegharkunik",
    "Գեղարքունիք": "Gegharkunik",
    "Գավառ": "Gegharkunik",
    "Մարտունի": "Gegharkunik",
    " Վայոց Ձոր ": "Vayots Dzor",
    "Վայոց Ձոր": "Vayots Dzor",
    "Ագարակաձոր": "Vayots Dzor",
    "Արենի": "Vayots Dzor",
    "Եղեգնաձոր": "Vayots Dzor",
    "Ջերմուկ": "Vayots Dzor",
    "Ռինդ": "Vayots Dzor",
    "Վայք": "Vayots Dzor",
    " Տավուշ ": "Tavush",
    "Տավուշ": "Tavush",
    "Հաղարծին": "Tavush",
    " Կոտայք ": "Kotayk",
    "Կոտայք": "Kotayk",
    "Աբովյան": "Kotayk",
    "Առինջ": "Kotayk",
    "Արզնի": "Kotayk",
    "Բյուրեղավան": "Kotayk",
    "Գետամեջ": "Kotayk",
    "Եղվարդ": "Kotayk",
    "Կարենիս": "Kotayk",
    "Հրազդան": "Kotayk",
    "Չարենցավան": "Kotayk",
    "Ջրվեժ": "Kotayk",
    "Քասախ": "Kotayk",
    " Լոռի ": "Lori",
    "Լոռի": "Lori",
    "Ալավերդի": "Lori",
    "Ջրաշեն": "Lori",
    "Սպիտակ": "Lori",
    "Վանաձոր": "Lori",
    "Օձուն": "Lori",
    " Շիրակ ": "Shirak",
    "Շիրակ": "Shirak",
    "Արթիկ": "Shirak",
    "Գյումրի": "Shirak",
    "Մարալիկ": "Shirak",
    "Փոքր Մանթաշ": "Shirak",
    " Սյունիք ": "Syunik",
    "Սյունիք": "Syunik",
    "Գորիս": "Syunik",
    "Կապան": "Syunik",
    "Քաջարան": "Syunik",
    "ԼՂՀ": "Artsakh"
}

data["Region"] = data["Region"].map(translation_dict)

In [34]:
# Dictionary for translating Work sphere
translation_dict = {
    " Առողջապահություն ": "Healthcare",
    " Գյուղատնտեսություն, անտառային տնտեսություն և ձկնորսություն ": "Agriculture, forestry and fishing",
    " Կացության և հանրային սննդի կազմակերպում ": "Organization of accomodation and food safety",
    " Էլեկտրականության, գազի, գոլորշու և լավորակ օդի մատակարարում ": "Supply of electricity, gas, steam and good quality air",
    " Կրթություն ": "Education",
    " Հանքագործական արդյունաբերություն և բացահանքերի շահագործում ": "Mining industry and open pit operations",
    " Մեծածախ և մանրածախ առևտուր. Ավտոմեքենաների և մոտոցիկլների նորոգում ": "Wholesale and retail trade. Car and motorcycle repair",
    " Մասնագիտական, գիտական և տեխնիկական գործունեություն ": "Professional, scientific and technical activities",
    " Պետական կառավարում և պաշտպանություն ": "Public administration and defense",
    " Մշակույթ, զվարճություն, հանգիստ ": "Culture, entertainment, relaxation",
    " Մշակող արդյունաբերություն ": "Manufacturing",
    " Տեղեկատվություն և կապ ": "Information and connection",
    " Վարչարարական և օժանդակ գործունեություն ": "Administartive and supportive activities",
    " Փոխադրումներ և պահեստային տնտեսություն ": "Transportation and warehousing",
    " Ֆինանսական և ապահովագրական գործունեություն ": "Financial and insurance activities",
    " Ֆիզ․անձ/ԱՁ/անհայտ ": "Individual entrepreneur",
    " Ջրամատակարարում, կոյուղի, թափոնների կառավարում և վերամշակում ": "Water supply, sewage, waste management and recycling",
}

data["Work sphere"] = data["Work sphere"].map(translation_dict)

In [35]:
# Dictionary for translating Sex
translation_dict = {
    " Անհայտ ": "Unknown",
    " Արական ": "Male",
    "Արական": "Male",
    " Իգական ": "Female",
    "Իգական": "Female"
}

data["Sex"] = data["Sex"].map(translation_dict)

In [36]:
data.head(70)

,N,Insured person Code,Work sphere,Start Date,End Date,DOB,Sex,Region,Claim Code,Claim Status,Illness/ICD Code,Illness/Category,Claim Amount,Contract Longevity,BirthYear,Age
0,1,QH6419321,Individual entrepreneur,2022-06-18,2023-06-17,14/10/2005,Female,Lori,GT2916303634,Satisfied,K02,Dentistry,18000.0,364 days,2005,19
1,2,IY5242264,Wholesale and retail trade. Car and motorcycle...,2022-01-07,2023-06-30,29/06/1988,Female,Yerevan,JU4564505110,Satisfied,"E55,Y44.1",Family doctor,37000.0,539 days,1988,36
2,3,CR2990565,"Professional, scientific and technical activities",2023-10-02,2024-09-02,25/10/1975,Female,Tavush,VD6289680601,Satisfied,"K02,K03.6,K04",Dentistry,45000.0,336 days,1975,49
3,4,CE9774993,Financial and insurance activities,2023-01-01,2023-12-31,4/10/1983,Female,Yerevan,KD9088354072,Satisfied,M54,Neurology,0.0,364 days,1983,41
4,5,EA2798820,Information and connection,2023-03-15,2024-03-14,28/04/1990,Male,Yerevan,GR2730567929,Satisfied,"M54.5,R25.2",Neurology,110960.0,365 days,1990,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,66,OE3903242,Information and connection,2023-01-02,2024-01-31,8/12/2004,Male,Yerevan,UZ2756656360,Canceled,\n(T79),Traumatology,0.0,394 days,2004,20
66,67,OC2258896,Financial and insurance activities,2023-01-01,2023-12-31,17/09/1994,Female,Yerevan,SQ4404958413,Satisfied,Z04,Allergology,3007.0,364 days,1994,30
67,68,ET8765050,Financial and insurance activities,2023-01-01,2023-12-31,15/09/1998,Unknown,Yerevan,PE4277929623,Claim creation,Z04,Oncology,0.0,364 days,1998,26
68,69,MT1801014,Information and connection,2022-01-10,2023-09-30,11/5/1999,Female,Yerevan,HX3534253601,Satisfied,E55.9,Family doctor,7000.0,628 days,1999,25


## Downloading the cleaned data and saving as a new csv file

In [37]:
data.to_csv('cleaned_data.csv', index = False) 

## Statistical analysis

### Descriptive statistics

In [39]:
desc_stats = data.describe()

cat_stats = data.describe(include=['object'])

print("Descriptive statistics for numerical columns:")
print(desc_stats)

print("\nSummary statistics for categorical columns:")
print(cat_stats)

Descriptive statistics for numerical columns:
                   N  Claim Amount           Contract Longevity  \
count  249755.000000  2.497550e+05                       249755   
mean   125067.128806  2.915239e+04  385 days 16:59:57.650421800   
std     72187.334046  7.082029e+04  117 days 23:59:03.634306500   
min         1.000000 -2.500000e+03          -332 days +00:00:00   
25%     62542.500000  8.000000e+03            337 days 00:00:00   
50%    125121.000000  1.500000e+04            364 days 00:00:00   
75%    187572.500000  3.000000e+04            394 days 00:00:00   
max    250033.000000  4.933260e+06            729 days 00:00:00   

           BirthYear            Age  
count  249755.000000  249755.000000  
mean     1988.324702      35.675298  
std        14.674593      14.674593  
min      1930.000000       0.000000  
25%      1981.000000      28.000000  
50%      1989.000000      35.000000  
75%      1996.000000      43.000000  
max      2024.000000      94.000000  

Summary

### Correlation analysis

In [41]:
corr_matrix = data.corr()

print("Correlation matrix:")
print(corr_matrix)

Correlation matrix:
                     N  Claim Amount  BirthYear       Age
N             1.000000     -0.014535  -0.068797  0.068797
Claim Amount -0.014535      1.000000  -0.066978  0.066978
BirthYear    -0.068797     -0.066978   1.000000 -1.000000
Age           0.068797      0.066978  -1.000000  1.000000


### Grouped analysis

In [43]:
region_mean_claim = data.groupby('Region')['Claim Amount'].mean()

print("Mean claim amount by region:")
print(region_mean_claim)

Mean claim amount by region:
Region
Aragatsotn     25151.633317
Ararat         27716.370055
Armavir        25872.933764
Artsakh        28104.226131
Gegharkunik    25986.911421
Kotayk         25934.171422
Lori           28517.681994
Shirak         28808.555624
Syunik         29601.977915
Tavush         25362.423184
Vayots Dzor    26837.892050
Yerevan        29445.130930
Name: Claim Amount, dtype: float64


### Frequency analysis

In [47]:
illness_category_counts = data['Illness/Category'].value_counts()

print("Frequency of each illness category:")
print(illness_category_counts)

Frequency of each illness category:
Dentistry                        42750
Family doctor                    30033
Ophthalmology                    25324
Annual medical examination       23514
Gynecology                       15164
ENT                              14732
Endocrinology                    13819
Pediatrics                       12420
Neurology                        11223
Dermatology                       8902
Traumatology                      7616
Cardiology                        7108
Gastroenterology                  6443
Urology                           4562
Pregnancy                         3845
General surgery                   3170
Oncology                          3008
Initial screening                 2430
Angiology                         2307
Rheumatology                      2295
Proctology                        1855
Vertebrology                      1492
Obstetrics                        1485
Allergology                       1163
Psychiatry                  

### Hypothesis testing

In [49]:
from scipy import stats

male_claims = data[data['Sex'] == 'Male']['Claim Amount']
female_claims = data[data['Sex'] == 'Female']['Claim Amount']

t_stat, p_value = stats.ttest_ind(male_claims, female_claims)

print("T-statistic:", t_stat)
print("P-value:", p_value)

T-statistic: 11.038529761384485
P-value: 2.5293791932199203e-28
